In [ ]:
import pandas as pd

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
%matplotlib inline 
matplotlib.rcParams['figure.figsize'] = (20,10)

In [ ]:
df = pd.read_csv('/kaggle/input/delhi-house-price-prediction/MagicBricks.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.groupby('Type')['Type'].agg('count')

In [ ]:
df.Bathroom.unique()

In [ ]:
df1 = df.drop(['Furnishing','Transaction','Type','Status'],axis = 'columns')

In [ ]:
len(df1)

In [ ]:
df1.isnull().sum()

In [ ]:
df2=df1.dropna()

In [ ]:
df2.info()

In [ ]:
df2.head()

In [ ]:
df.Bathroom.unique()

In [ ]:
df3 = df2[~(df2.Bathroom >4)]

In [ ]:
df3.info()

In [ ]:
round(df3.Area)

In [ ]:
df3.Area.unique()

In [ ]:
df3.head()

In [ ]:
df2.Per_Sqft.unique()

In [ ]:
float('5128')

In [ ]:
df3[df3.BHK < df3.Bathroom]

In [ ]:
df4 = df3[~(df3.Parking >4)]

In [ ]:
df4.info()

In [ ]:
location_stat = df4.groupby(['Locality'])['Locality'].agg('count').sort_values(ascending = False)

In [ ]:
location_stat_less_than4 = location_stat[location_stat ==2]

In [ ]:
len(location_stat_less_than4)

In [ ]:
df4.Locality = df4['Locality'].apply(lambda x : 'other' if x in location_stat_less_than4 else x)

In [ ]:
len(df4[df4.Locality=='other'])

In [ ]:
df4.groupby(['Locality'])['Locality'].agg('count').sort_values(ascending = False)

In [ ]:
df4.info()

In [ ]:
df4.Locality.unique()

In [ ]:
df4[df4.Locality == 'Tara Apartment, Alaknanda carpet area 1400 sqft status Ready to Move floor 2 out of 3 floors transaction Resale furnishing Furnished facing North - East overlooking Garden/Park society Tara Apartment car parking 1 Covered, 1 Open bathroom 3 balcony 4 ownership Freehold Near Alaknanda Main Market, Jamia Humderd University & Batra hospital. A very good 3 bhk flat for sale in rwa Tara Apartment, delhi south. It is a superb property and offers an excellent view. The flat is unfurnished. It promises a comfortable stay. It is feng shuivaastu compliant, which is considered to bring positive energy. The appeal of the flat has been increased with the availability of a pipedGa. Indeed, the society too has multiple facilities for enjoyment, such as club housecommunity center and fitness centregym etc. It also has rain water harvesting system. It has a super builtUp area of 180 sq. Yards. Other f read more Contact Agent View Phone No. Share Feedback Agent Sulabh Estate']

In [ ]:
df5 = df4[~(df4.Area/df4.BHK <300)]

In [ ]:
df5

In [ ]:
df4[(df4.Area/df4.BHK <300)]

In [ ]:
df5.Per_Sqft.describe()

In [ ]:
df6 = df5[~(df5.Per_Sqft < 2500)]

In [ ]:
df6.Per_Sqft.describe()

In [ ]:
df6.info()

In [ ]:
plt.hist(df6.Bathroom, width = 0.3)
plt.xlabel('Number of Bathroom')
plt.ylabel('Count')

In [ ]:
plt.hist(df6.Parking, width = 0.3)
plt.xlabel('Number of Bathroom')
plt.ylabel('Count')

In [ ]:
df6[df6.BHK < (df6.Parking)]

In [ ]:
df6.head()

In [ ]:
df7 = df6.drop(['Per_Sqft'],axis = 'columns')

In [ ]:
df7.head()

In [ ]:

# convert integers to floats
df7['BHK'] = df7['BHK'].astype(float)
df7['Price'] = df7['Price'].astype(float)


In [ ]:
#convert locality into catagories

In [ ]:
print(df7.dtypes)

In [ ]:
df7.to_csv('df7.csv')

In [ ]:
dummies = pd.get_dummies(df7.Locality,dtype=float)

In [ ]:
dummies

In [ ]:
df8 = pd.concat([df7,dummies.drop('other',axis='columns')],axis = 'columns')

In [ ]:
df8

In [ ]:
df9 = df8.drop('Locality',axis = 'columns')

In [ ]:
df9.head()

In [ ]:
x= df9.drop('Price',axis = 'columns')

In [ ]:
y= df9.Price

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train, y_test = train_test_split(x,y,test_size= 0.2, random_state =10)

In [ ]:
# print(x_train)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lnr = LinearRegression()

In [ ]:
x_train = x_train.fillna(x_train.mean())
y_train= y_train.fillna(y_train.mean())
x_test= x_test.fillna(x_train.mean())
y_test = y_test.fillna(y_train.mean())
x = x.fillna(x.mean())
y = y.fillna(y.mean())



In [ ]:
lnr.fit(x_train,y_train)

In [ ]:
# y_pred = lnr.predict(x_test)

In [ ]:
lnr.score(x_test,y_test)

In [ ]:
from sklearn.model_selection import ShuffleSplit

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cv = ShuffleSplit(n_splits =5,test_size =0.2, random_state =0)
cross_val_score(LinearRegression(),x,y,cv=cv)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

In [ ]:
def find_model_using_grid_search(x,y):
    algos = {
        'linear_regression': {
            'model' : LinearRegression(),
            'params': {
                'normalize': [True,False]
            }
        },
        'lasso' :{
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random','cyclic']
            }
        },
        'decision_tree': {
            'model' : DecisionTreeRegressor(),
            'params': {
                'criterion': ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
        
        
    }
    score = []
    cv = ShuffleSplit(n_splits =5,test_size =0.2, random_state =0)
    for algo_name, config in algos.items():
        gs = GridSearchCV(config['model'],config['params'],cv=cv,return_train_score=False)
        gs.fit(x,y)
        score.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })
    return pd.DataFrame(score,columns= ['model','best_score','best_params'])

In [ ]:
find_model_using_grid_search(x,y)

In [ ]:
def predict_price(location,sqft,bath,bhk,park):
    loc_index = np.where(x.columns==location)[0][0]
    
    X= np.zeros(len(x.columns))
    X[0]= sqft
    X[1] = bath
    X[2] = bhk
    X[3] = park
    if loc_index >= 0:
        X[loc_index] =1
        
    return lnr.predict([X])[0]

In [ ]:
predict_price('Rohini Sector 24',1300,3,4,1)